In [1]:
import pyspark
from pyspark.sql import SparkSession
app_name = "bubbly"
master = "local[*]"
spark = (SparkSession.builder
    .master(master)
    .config("spark.driver.cores", 1)
    .appName(app_name)
    .getOrCreate() )
sc = spark.sparkContext
print ('SparkContext created')

SparkContext created


In [3]:
from pyspark.sql.types import StructType, StructField, IntegerType, DateType, StringType
accidenteData = spark.read.csv ('hdfs://localhost:9000/user/ubuntu/accidentes/2018.csv',header=True)
#accidenteData = spark.read.csv ('file:///home/ubuntu/Downloads/2018_Accidentalidad.csv',header=True)


In [30]:
import pyspark.sql.functions as func

accRed=accidenteData.select(
        func.to_date(func.concat_ws(accidenteData.FECHA,' 00:00:00'), 'dd/MM/yyyy HH:mm:ss').alias('fecha'), \
        accidenteData.DISTRITO.alias('distrito'), \
        accidenteData["Nº PARTE"].alias('idAccidente'), \
        func.substring(accidenteData.LESIVIDAD,1,2).alias('lesividad'), \
)

accPorDia=accRed.filter(accRed.lesividad != "IL")\
                .groupBy('fecha','distrito','idAccidente','lesividad')\
                .agg(func.count('lesividad').alias('victimas'))\
                .sort ('fecha','distrito','idAccidente','lesividad')

AnalysisException: "cannot resolve '` 00:00:00`' given input columns: [FECHA, Nº, Tramo Edad, LESIVIDAD, CPFA Seco, CPSV Grava Suelta, SEXO, CPSV Mojada, TIPO ACCIDENTE, CPSV Seca Y Limpia, CPFA Nieve, LUGAR ACCIDENTE, CPFA Niebla, DIA SEMANA, CPFA Granizo, CPFA Hielo, * Nº VICTIMAS, TIPO PERSONA, CPSV Hielo, Nº PARTE, CPFA Lluvia, DISTRITO, Tipo Vehiculo, CPSV Aceite, CPSV Barro, RANGO HORARIO];;\n'Project [to_date(concat(FECHA#10, ' 00:00:00), Some(dd/MM/yyyy HH:mm:ss)) AS fecha#263, DISTRITO#13 AS distrito#264, Nº PARTE#16 AS idAccidente#265, substring(LESIVIDAD#34, 1, 2) AS lesividad#266]\n+- Relation[FECHA#10,RANGO HORARIO#11,DIA SEMANA#12,DISTRITO#13,LUGAR ACCIDENTE#14,Nº#15,Nº PARTE#16,CPFA Granizo#17,CPFA Hielo#18,CPFA Lluvia#19,CPFA Niebla#20,CPFA Seco#21,CPFA Nieve#22,CPSV Mojada#23,CPSV Aceite#24,CPSV Barro#25,CPSV Grava Suelta#26,CPSV Hielo#27,CPSV Seca Y Limpia#28,* Nº VICTIMAS#29,TIPO ACCIDENTE#30,Tipo Vehiculo#31,TIPO PERSONA#32,SEXO#33,... 2 more fields] csv\n"

In [27]:
accPorDia.toPandas().to_csv('/home/ubuntu/Downloads/victimas_por_fecha_distrito_accidente_lesion.csv',sep=';')

In [25]:
func.to_date("31/12/2018 00:00:00").collect()

TypeError: 'Column' object is not callable

In [ ]:
func.